In [1]:
import os
import pandas as pd
import numpy
import pickle

In [2]:
with open('vrabes_beh_dict.pkl','rb') as f:
    total = pickle.load(f)

In [3]:
def ses_per_sub (df):        
    ses_dict = {}
    ses_dict2 = {}
    for i in df['ID']:
        sub = i.split('_')[0]
        ses = i.split('_')[1]
        if sub in ses_dict.keys():
            ses_dict[sub] = ses_dict[sub]+1
        else: ses_dict[sub] = 1
    
    for a in df['ID']:
        sub = a.split('_')[0]
        ses_dict2[a] = ses_dict[sub]    

    return ses_dict2

In [4]:
def time_ext (df):
    cols = []    
    for col in df.columns:
        if col.split('_')[-1] == 'Start Time':
            cols.append(col)
        else: pass
    cols.insert(0,'ID')        
    return df[cols]

In [5]:
def as_ext(df):
    cols = []
    for col in df.columns:
        if "AnxScr" in col.split('_'):        
            cols.append(col)
        else: pass
    cols.insert(0,'ID')
    return df[cols]

In [6]:
def hrv_result(df):
    cols = []
    for col in df.columns:
        if "Result" in col.split('_'):
            cols.append(col)
        else: pass
    cols.insert(0,'ID')
    return df[cols]

In [7]:
def crf_ext(df,ses):
    cols = []
    for col in df.columns:
        if ses in col.split('_'):
            cols.append(col)
        else:pass
    cols.insert(0,'ID')
    locals()['ses_{}_crf'.format(ses.split('-')[-1])] = df[cols]
    return locals()['ses_{}_crf'.format(ses.split('-')[-1])]

In [8]:
def demo_ext(df,ses):
    cols = []
    if '_'.join([ses,'enroll_d_y']) in df.columns:
        demo_start = df.columns.get_loc('_'.join([ses,'enroll_d_y']))
    else: demo_start = df.columns.get_loc('_'.join([ses,'enroll_d']))
    demo_last = df.columns.get_loc('_'.join([ses,'alcohol4']))
    cols = list(df.columns[demo_start:demo_last])
    cols.insert(0,'ID')
    return df[cols]


In [9]:
def extractor(df):
    session = ['ses-01','ses-02','ses-03','ses-04']
    extracted = {}
    for i in session:               
        ses = df.loc[df['ID'].str.contains(i),:]        
        demo = demo_ext(ses,i)
        anxscr = as_ext(ses)
        hrv = hrv_result(ses)
        time = time_ext(ses)
        crf = crf_ext(ses,i)
        anxscr = anxscr.loc[:,anxscr.notnull().any()]
        hrv = hrv.loc[:,hrv.notnull().any()]
        time = time.loc[:,time.notnull().any()]
        crf = crf.loc[:,crf.notnull().any()]
        extracted[i] = [demo, anxscr,hrv, time,crf]
    return extracted

  

In [10]:
extracted = extractor(total)
with open('vrabes_extracted.pkl','wb') as f:
    pickle.dump(extracted,f)  